In [1]:
import pandas as pd
pd.set_option("display.max_colwidth", 40)
import numpy as np
import re
import spacy
import nltk
from nltk import FreqDist
import gensim
from gensim import corpora

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Load  Data

In [4]:
data = pd.read_csv("comments_lemmentized.csv", sep=";", index_col=0)
#data1 = pd.read_csv("center_parcs_clean_concatenated.csv")
#data1 = pd.read_csv('clean_data_gr4.csv',index_col=0)

In [3]:
data = pd.read_csv('clean_data_gr4.csv', index_col = None)
data.drop(['Unnamed: 0'], axis = 1)

FileNotFoundError: [Errno 2] File b'clean_data_gr4.csv' does not exist: b'clean_data_gr4.csv'

In [5]:
type(data)
data = pd.DataFrame(data1['Stemming'])

NameError: name 'data1' is not defined

In [11]:
data2

AttributeError: 'LazyConfigValue' object has no attribute 'lower'

                          lemmantized_reviews
0     ['premium', 'cottage', 'renovate', '...
1     ['nice', 'environment', 'peaceful', ...
2     ['expectation', 'high', 'read', 'rev...
3     ['quite', 'center', 'parcs', 'france...
4     ['new', 'year', 'trip', 'family', 's...
5     ['visit', 'different', 'centre', 'pa...
6     ['read', 'review', 'sceptical', 'par...
7     ['anyone', 'look', 'book', 'center',...
8     ['unfortunately', 'experience', 'tot...
9     ['enjoyable', 'stay', 'village', 'lo...
10    ['park', 'okay', 'good', 'time', 'an...
11    ['stay', 'centerparcs', 'best', 'cou...
12    ['aqua', 'dome', 'good', 'wide', 'va...
13    ['location', 'set', 'site', 'perfect...
14    ['holiday', 'family', 'child', 'frie...
15    ['often', 'drive', 'mile', 'want', '...
16    ['weu2019ve', 'return', 'week', 'cen...
17    ['read', 'trip', 'advisor', 'review'...
18    ['staff', 'brilliant', 'others', 'aw...
19    ['first', 'trip', 'france', 'family'...
20    ['stay', 'week', 'recommenda

In [6]:
data

AttributeError: 'LazyConfigValue' object has no attribute 'lower'

                          lemmantized_reviews
0     ['premium', 'cottage', 'renovate', '...
1     ['nice', 'environment', 'peaceful', ...
2     ['expectation', 'high', 'read', 'rev...
3     ['quite', 'center', 'parcs', 'france...
4     ['new', 'year', 'trip', 'family', 's...
5     ['visit', 'different', 'centre', 'pa...
6     ['read', 'review', 'sceptical', 'par...
7     ['anyone', 'look', 'book', 'center',...
8     ['unfortunately', 'experience', 'tot...
9     ['enjoyable', 'stay', 'village', 'lo...
10    ['park', 'okay', 'good', 'time', 'an...
11    ['stay', 'centerparcs', 'best', 'cou...
12    ['aqua', 'dome', 'good', 'wide', 'va...
13    ['location', 'set', 'site', 'perfect...
14    ['holiday', 'family', 'child', 'frie...
15    ['often', 'drive', 'mile', 'want', '...
16    ['weu2019ve', 'return', 'week', 'cen...
17    ['read', 'trip', 'advisor', 'review'...
18    ['staff', 'brilliant', 'others', 'aw...
19    ['first', 'trip', 'france', 'family'...
20    ['stay', 'week', 'recommenda

In [7]:
data.head()

AttributeError: 'LazyConfigValue' object has no attribute 'lower'

                       lemmantized_reviews
0  ['premium', 'cottage', 'renovate', '...
1  ['nice', 'environment', 'peaceful', ...
2  ['expectation', 'high', 'read', 'rev...
3  ['quite', 'center', 'parcs', 'france...
4  ['new', 'year', 'trip', 'family', 's...

In [8]:
#the lemmantized_reviews are interpreted as a string by the read_csv command. We convert it into a list. 
data['Stemming'] = data['Stemming'].str.replace(r"[\[\]\']", "").str.split(", ")

KeyError: 'Stemming'

In [9]:
data['Stemming'][0][:10] #checking if it's a list

KeyError: 'Stemming'

In [10]:
data.dropna(subset=['Stemming'], inplace=True)

KeyError: ['Stemming']

In [11]:
data.shape

(1303, 1)

## Remove words

In [32]:
# We list the words we want to get rid of
list_stop_words = ["good", "great", "one", "nt", "u2013", "", "would", "get", "center", "parcs","the","le"
                   ," u00e0","nous","with","what","for","etait","not","the","une","de","pour","plus","zip","under","but","tr u00e0","   ",
                  'n\'t','...','\'ve','\'re', 'ã\x9dou', "''", 'avez','est','être','être.on','le','les','de','et','la','pour','très','à',
                'en','a','des','pas','dans','du','au','avec','sont','que','avons','de','et','pour'
                ,'nous','un','est','en','des','pas','dans','du','au','avec','-\\taqua','-\\ten','-\\tle','farfelues.moi','\\u2764','amp','dis','\\u00bc',
               'tré','tres','-\\tpas','-\\tla','1m01','cafetière\\u2026','une','tout','mais','qui','il','sur',"c'est",'ne','plus','ce','se',
               'aux','je','peu','vraiment','par','vous','fait','faire','était','car','ou','quot','si','sans','tous','aussi','fois','n','ans','entre','peut',
               'semaine','faut','voir','3','eu','cela','ont','nos','4','même','nos','contre','donc','surtour','son','reste','séjour','site','rapport',
                'surtout','quand','2','sommes','sinon','vu','jours','assez','autres','\u00e7a','ménage',"n'est",'toujours','soit','c','l','malgré','comme',
                'niveau','toutes','non','avions','toute','\u00e7a','mon','moins','passer','devant','où',"c est","trop"," u00e7a","parc","daims","cottage",
                   "sejour","etre","mundo","rien",'aqua',"bois","cottages","passe","l aqua","d activites","n'est","j'ai",'avant','apres','ils','alors',
                   'trouve','deja',"qu il",'dit','vaut','enfin','quoi','ceux','cela','c']

In [18]:
def remove_stop_word(x, list_stop_words=list_stop_words):
    clean_x = [l for l in x if l not in list_stop_words]
    return clean_x

In [19]:
data['Stemming'] = data['Stemming'].apply(remove_stop_word)

In [20]:
data.shape

(1994, 1)

## Building an LDA model

In [21]:
dictionary = corpora.Dictionary(data['Stemming'])

In [22]:
len(dictionary)

12373

In [23]:
doc_term_matrix = [dictionary.doc2bow(rev) for rev in data['Stemming']]

In [24]:
import warnings
warnings.filterwarnings('ignore')
import re
import matplotlib.pyplot as plt

# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
num_topics = 5
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, 
                num_topics=num_topics, 
                alpha=[0.001] * num_topics, 
                eta=[0.001] * len(dictionary),
                chunksize=1000,
                passes=20,
                random_state=100,
               )

In [25]:
lda_model.print_topics(num_words=8)

[(0,
  '0.011*"service" + 0.009*"sale" + 0.006*"vaisselle" + 0.006*"enfants" + 0.006*"prix" + 0.005*"point" + 0.005*"agreable" + 0.005*"arrivee"'),
 (1,
  '0.012*"activites" + 0.012*"personnel" + 0.009*"vip" + 0.008*"calme" + 0.008*"nature" + 0.008*"propre" + 0.008*"famille" + 0.008*"agreable"'),
 (2,
  '0.012*"enfants" + 0.011*"bien" + 0.011*"activites" + 0.008*"personnel" + 0.008*"prix" + 0.007*"grand" + 0.007*"piscine" + 0.006*"famille"'),
 (3,
  '0.027*"bien" + 0.013*"enfants" + 0.011*"activites" + 0.009*"agreable" + 0.009*"famille" + 0.009*"super" + 0.008*"piscine" + 0.007*"propre"'),
 (4,
  '0.019*"bien" + 0.014*"activites" + 0.012*"agreable" + 0.012*"enfants" + 0.010*"nature" + 0.010*"famille" + 0.007*"prix" + 0.007*"vip"')]

## Topic visualization

In [27]:
dictionary

In [29]:
# Visualize the topics
pd.options.display.max_colwidth = 1000
viz = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary, mds='tsne')


In [31]:
pyLDAvis.enable_notebook()
viz
pyLDAvis.show(viz)


Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [11/Feb/2019 15:00:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2019 15:00:26] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2019 15:00:26] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2019 15:00:26] code 404, message Not Found
127.0.0.1 - - [11/Feb/2019 15:00:26] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/Feb/2019 15:00:26] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2019 15:04:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2019 15:04:08] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2019 15:04:08] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2019 15:04:08] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2019 15:04:08] code 404, message Not Found
127.0.0.1 - - [11/Feb/2019 15:04:08] "GET /favicon.ico HTTP/1.1" 404 -



stopping Server...
